In [2]:
import argparse
import os
import numpy as np
import multiprocessing as mp
import polars as pl
import queue
from byu.data.io import MultithreadOpencvTomogramLoader
from torch.nn import functional as F
import torch
import random
from tqdm import tqdm

In [3]:
df = pl.scan_csv('/home/dangnh36/datasets/.comp/byu/processed/gt_v2.csv').collect()
df

tomo_id,Z,Y,X,voxel_spacing,ori_num_motors,num_motors,ZA,YA,XA,V,VA,motor_z,motor_y,motor_x,motor_zA,motor_yA,motor_xA,motor_zyx,motor_zyxA
str,i64,i64,i64,f64,i64,i64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,str,str
"""tomo_003acc""",500,1912,1847,6.5,0,0,3250.0,12428.0,12005.5,1765732000,4.8491e11,-1.0,-1.0,-1.0,-6.5,-6.5,-6.5,"""[]""","""[]"""
"""tomo_00e047""",300,959,928,15.6,1,1,4680.0,14960.4,14476.8,266985600,1.0136e12,169.0,546.0,603.0,2636.4,8517.6,9406.8,"""[[169.0, 546.0, 603.0]]""","""[[2636.4, 8517.6, 9406.8]]"""
"""tomo_00e463""",500,924,956,19.7,6,6,9850.0,18202.8,18833.2,441672000,3.3767e12,235.0,403.0,137.0,4629.5,7939.1,2698.9,"""[[235.0, 403.0, 137.0], [243.0…","""[[4629.5, 7939.099999999999, 2…"
"""tomo_01a877""",300,960,928,13.1,1,1,3930.0,12576.0,12156.8,267264000,6.0083e11,147.0,638.0,286.0,1925.7,8357.8,3746.6,"""[[147.0, 638.0, 286.0]]""","""[[1925.7, 8357.8, 3746.6]]"""
"""tomo_02862f""",300,959,928,15.6,1,1,4680.0,14960.4,14476.8,266985600,1.0136e12,101.0,351.0,120.0,1575.6,5475.6,1872.0,"""[[101.0, 351.0, 120.0]]""","""[[1575.6, 5475.599999999999, 1…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""tomo_fe050c""",300,959,928,15.6,1,2,4680.0,14960.4,14476.8,266985600,1.0136e12,138.0,134.0,542.0,2152.8,2090.4,8455.2,"""[[138.0, 134.0, 542.0], [143.1…","""[[2152.7999999999997, 2090.4, …"
"""tomo_fe85f6""",800,928,960,13.1,0,1,10480.0,12156.8,12576.0,712704000,1.6022e12,463.5,903.5,747.0,6071.85,11835.85,9785.7,"""[[463.5, 903.5, 747.0]]""","""[[6071.849999999999, 11835.85,…"
"""tomo_fea6e8""",300,960,928,16.8,0,0,5040.0,16128.0,15590.4,267264000,1.2673e12,-1.0,-1.0,-1.0,-16.8,-16.8,-16.8,"""[]""","""[]"""


In [4]:
tomo_loader = MultithreadOpencvTomogramLoader(8)

In [5]:
from copy import deepcopy
import cv2

In [ ]:
device = torch.device('cpu')

os.makedirs('/home/dangnh36/datasets/.comp/byu/processed/pseudo_test/tomograms', exist_ok=False)

new_rows = []
for row_idx, row in enumerate(df.sample(20).iter_rows(named = True)):
    print(row)
    tomo_id = row['tomo_id']

    tomo_dir = os.path.join('/home/dangnh36/datasets/.comp/byu/raw/train/', tomo_id)

    # read ori tomo
    tomo = tomo_loader.load(tomo_dir)
    ori_shape = tomo.shape  # ZYX
    print(ori_shape)
    MAX_SHAPE = (450, 1500, 1200)
    max_new_shape = [(random.randrange(int(new * 1.2), int(new*1.5))) for new in MAX_SHAPE]
    print('max new shape:', max_new_shape)
    scale = max([ new/old for new, old in zip(max_new_shape, ori_shape)])
    print('scale:', scale)

    new_tomo = F.interpolate(
            torch.from_numpy(tomo).to(device)[None, None].float(),
            size=None,
            scale_factor=scale,
            mode='trilinear',
            align_corners=None,
            recompute_scale_factor=False,
    )[0, 0]
    new_tomo = torch.clip(new_tomo, 0, 255.0).to(torch.uint8).cpu().numpy()
    print('Return', new_tomo.shape, new_tomo.dtype)

    # scale the annotations too
    new_row = deepcopy(row)
    new_row['Z'] = new_tomo.shape[0]
    new_row['Y'] = new_tomo.shape[1]
    new_row['X'] = new_tomo.shape[2]
    new_row['voxel_spacing'] /= scale
    new_row['V'] *= (scale ** 3)
    if row['motor_z'] != -1:
        new_row['motor_z'] *= scale
        new_row['motor_y'] *= scale
        new_row['motor_x'] *= scale
    new_rows.append(new_row)
    print('-----------------------------------\n\n')

    tomo_dir = f'/home/dangnh36/datasets/.comp/byu/processed/pseudo_test/tomograms/{tomo_id}'
    os.makedirs(tomo_dir, exist_ok=False)
    for z_idx in tqdm(range(new_tomo.shape[0]), desc = f'Saving {tomo_id}'):
        slice_img = new_tomo[z_idx]
        fname = os.path.join(tomo_dir, f"slice_{z_idx:04d}.jpg")
        cv2.imwrite(fname, slice_img)

new_df = pl.DataFrame(new_rows)
new_df = new_df.select(pl.col(['tomo_id', 'Z', 'Y', 'X', 'voxel_spacing', 'ori_num_motors', 'num_motors', 'motor_z', 'motor_y', 'motor_x']))
new_df.write_csv('/home/dangnh36/datasets/.comp/byu/processed/pseudo_test/gt.csv')

{'tomo_id': 'tomo_a84050', 'Z': 300, 'Y': 928, 'X': 928, 'voxel_spacing': 13.1, 'ori_num_motors': 2, 'num_motors': 2, 'ZA': 3930.0, 'YA': 12156.8, 'XA': 12156.8, 'V': 258355200, 'VA': 580805999923.2, 'motor_z': 80.0, 'motor_y': 771.0, 'motor_x': 150.0, 'motor_zA': 1048.0, 'motor_yA': 10100.1, 'motor_xA': 1965.0, 'motor_zyx': '[[80.0, 771.0, 150.0], [150.0, 458.0, 606.0]]', 'motor_zyxA': '[[1048.0, 10100.1, 1965.0], [1965.0, 5999.8, 7938.599999999999]]'}
(300, 928, 928)
max new shape: [592, 1886, 1785]
scale: 2.0323275862068964
Return (609, 1885, 1885) uint8
-----------------------------------




Saving tomo_a84050: 100%|████████████████████████████████████████████████████████████████████| 609/609 [00:07<00:00, 81.10it/s]


{'tomo_id': 'tomo_ec607b', 'Z': 500, 'Y': 928, 'X': 960, 'voxel_spacing': 13.1, 'ori_num_motors': 2, 'num_motors': 2, 'ZA': 6550.0, 'YA': 12156.8, 'XA': 12576.0, 'V': 445440000, 'VA': 1001389655040.0, 'motor_z': 432.0, 'motor_y': 159.0, 'motor_x': 202.0, 'motor_zA': 5659.2, 'motor_yA': 2082.9, 'motor_xA': 2646.2, 'motor_zyx': '[[432.0, 159.0, 202.0], [466.0, 458.0, 301.0]]', 'motor_zyxA': '[[5659.2, 2082.9, 2646.2], [6104.599999999999, 5999.8, 3943.1]]'}
(500, 928, 960)
max new shape: [635, 2094, 1757]
scale: 2.2564655172413794
Return (1128, 2094, 2166) uint8
-----------------------------------




Saving tomo_ec607b: 100%|██████████████████████████████████████████████████████████████████| 1128/1128 [00:21<00:00, 53.53it/s]


{'tomo_id': 'tomo_0f9df0', 'Z': 300, 'Y': 960, 'X': 928, 'voxel_spacing': 15.6, 'ori_num_motors': 1, 'num_motors': 1, 'ZA': 4680.0, 'YA': 14976.0, 'XA': 14476.8, 'V': 267264000, 'VA': 1014645325824.0, 'motor_z': 94.0, 'motor_y': 174.0, 'motor_x': 599.0, 'motor_zA': 1466.3999999999999, 'motor_yA': 2714.4, 'motor_xA': 9344.4, 'motor_zyx': '[[94.0, 174.0, 599.0]]', 'motor_zyxA': '[[1466.3999999999999, 2714.4, 9344.4]]'}
(300, 960, 928)
max new shape: [647, 2007, 1644]
scale: 2.1566666666666667
Return (647, 2070, 2001) uint8
-----------------------------------




...

{'tomo_id': 'tomo_56b9a3', 'Z': 300, 'Y': 928, 'X': 928, 'voxel_spacing': 13.1, 'ori_num_motors': 1, 'num_motors': 1, 'ZA': 3930.0, 'YA': 12156.8, 'XA': 12156.8, 'V': 258355200, 'VA': 580805999923.2, 'motor_z': 60.0, 'motor_y': 299.0, 'motor_x': 498.0, 'motor_zA': 786.0, 'motor_yA': 3916.9, 'motor_xA': 6523.8, 'motor_zyx': '[[60.0, 299.0, 498.0]]', 'motor_zyxA': '[[786.0, 3916.9, 6523.8]]'}
(300, 928, 928)
max new shape: [658, 2026, 1514]
scale: 2.1933333333333334
Return (658, 2035, 2035) uint8
-----------------------------------




Saving tomo_56b9a3: 100%|████████████████████████████████████████████████████████████████████| 658/658 [00:08<00:00, 73.72it/s]


{'tomo_id': 'tomo_b0ded6', 'Z': 500, 'Y': 924, 'X': 956, 'voxel_spacing': 19.7, 'ori_num_motors': 2, 'num_motors': 2, 'ZA': 9850.0, 'YA': 18202.8, 'XA': 18833.2, 'V': 441672000, 'VA': 3376747183656.0, 'motor_z': 267.0, 'motor_y': 343.0, 'motor_x': 834.0, 'motor_zA': 5259.9, 'motor_yA': 6757.099999999999, 'motor_xA': 16429.8, 'motor_zyx': '[[267.0, 343.0, 834.0], [247.0, 335.0, 838.0]]', 'motor_zyxA': '[[5259.9, 6757.099999999999, 16429.8], [4865.9, 6599.5, 16508.6]]'}
(500, 924, 956)
max new shape: [672, 1851, 1751]
scale: 2.0032467532467533
Return (1001, 1851, 1915) uint8
-----------------------------------




Saving tomo_b0ded6: 100%|██████████████████████████████████████████████████████████████████| 1001/1001 [00:13<00:00, 72.89it/s]


{'tomo_id': 'tomo_b98cf6', 'Z': 800, 'Y': 928, 'X': 960, 'voxel_spacing': 13.1, 'ori_num_motors': 0, 'num_motors': 0, 'ZA': 10480.0, 'YA': 12156.8, 'XA': 12576.0, 'V': 712704000, 'VA': 1602223448063.9998, 'motor_z': -1.0, 'motor_y': -1.0, 'motor_x': -1.0, 'motor_zA': -13.1, 'motor_yA': -13.1, 'motor_xA': -13.1, 'motor_zyx': '[]', 'motor_zyxA': '[]'}
(800, 928, 960)
max new shape: [564, 2210, 1471]
scale: 2.3814655172413794


In [7]:
!rm -rf /home/dangnh36/datasets/.comp/byu/processed/pseudo_test/